# Resolución de problemas tipo "stiff"

<ul id="top">
</ul>

In [ ]:
using DifferentialEquations
using BenchmarkTools
using LinearAlgebra
using Plots

## 1 -  El problema del péndulo con muelle


Vamos a integrar numéricamente un problema de valor inicial que
corresponde a las ecuaciones del movimiento de un péndulo
formado por dos piezas unidas por un muelle que
se mueve en un plano vertical bajo la influencia de la fuerza de
la gravedad.



<img src="http://www.ehu.eus/ccwmuura/irakaskuntza/sinum/irudiak/pendulo.jpg" alt="pendulo" width="300" >




Este sistema mecánico tiene dos grados de libertad, y podemos
tomar como variables de posición del sistema el ángulo
$a$ y la distancia $L$ entre el punto de giro y el centro de
gravedad de la pieza inferior. Las ecuaciones del movimiento
correspondientes (bajo condiciones ideales de ausencia de
rozamiento) son las siguientes.

\begin{eqnarray*}
\frac{d^2}{dt^2} L &=& L \, \left(\frac{d a}{dt}\right)^2 + g \cos(a) - C_1 (L-L_0), \\
\frac{d^2}{dt^2} a &=& -\frac{g (C_2 + L) \sin(a) + 2 \, L \, \frac{d L}{dt}
\, \frac{d a}{dt}}{C_3 + L^2},
\end{eqnarray*}

donde $g$ es la aceleración de la gravedad, $L_0$ es el valor
de la distancia $L$ correspondiente al estado de reposo del
muelle,

\begin{eqnarray*}
C_1 = \frac{K}{m_2}, \quad C_2 = \frac{r m_1}{m_2}, \quad
C_3 = C_2 r + \frac{I_1+I_2}{m_2}, 
\end{eqnarray*}

$K$ es una constante determinada por la rigidez 
del muelle, $m_1$ y $m_2$ son
respectivamente las masas de la pieza superior e inferior,
$I_1$ e $I_2$ son los momentos de inercia de cada una de las
piezas respecto a su centro de gravedad, y $r$ es la distancia
entre el punto de giro y el centro de gravedad de la pieza
superior.

  Vamos a considerar el caso en que $L_0=1$, 
$g=1$, y las constantes $r$, $m_i,I_i$ (i=1,2) 
son tales que $C_2=1$ y $C_3=1$. Dependiendo de la rigidez del
muelle (dada por la constante $K$), se tendrán distintos
valores de la constante $C_1$. Consideraremos dos casos, que de
menor a mayor rigidez del muelle, serán $C_1 = 100$ y $C_1 = 10000$. En el segundo caso, tendremos un problem de EDOs de tipo "stiff".


Queremos
resolver el problema de valor inicial correspondiente a

$$
a(0)=\pi/2, \quad 
%d(0)=1+10^{-6} g \cos a(0), 
L(0)=L_0, 
\quad \frac{d}{dt} a(0)=0, \quad  \frac{d}{dt} L(0)=0,
$$

en el intervalo $t\in[0,40]$. 


A diferencia de las tareas de las semanas anteriores, en lugar de utilizar nuestras propias implementaciones de dichos algoritmos, utilizaremos la implementación disponibles en el paquete de resolución numérica de sistemas de EDOs disponible en Julia (DifferentialEquations.jl):

Haremos uso de dos algoritmos de integración numérica de sistemas de EDOs:
1. Una implementación de paso variable del método de Runge-Kutta de orden 5 de Dormand y Prince, DP5 (La función DOPRI5 implementada en la semana 5 es una versión simplificada de la misma.)
2. El método implícito de RK de orden 5 de Hairer (Hairer5), que como en el caso del método de Radau de orden 5 mostrado en las transparencias del Tema 4, es un **método apropiado para problems de tipo "stiff".**

Para ello, se ha de llamar a la función solve(), indicándole el método a utilizar (DP5() of Hairer5()), y la tolerancia absoluta (abstol) y tolerancia relativa (reltol) al error local. 
Los algoritmos a utilizar emplean una discretización temporal no uniforme adaptada al problema.

En todas las pruebas que hagamos con los distintos integradores,
variaremos la precisión de la integración variando el valor
de $tol$, y tomando tanto 'abstol' como 'reltol' igual a $tol$.

## 2 - Definición del sistema de EDOs

-  Para empezar, hay que escribir el sistema de arriba como un sistema de EDOs de primer orden de la forma du/dt = f(t,u, p), y posteriormente, **definir en Julia la función fmuelle(u,p,t) tal que f(t,u,p) = fmuelle(u, p, t).**

In [ ]:
function fmuelle(u,p,t)
  
    ?
    
    return ?
end

- **Evaluar dicha función para fmuelle(u,p,0.), donde u es el vector de estados correspondiente a los valores $a=1$, $L=2$, $da/dt=3$, $dL/dt=4$, y p es el vector de parámetros constantes correspondiente a $g=5$, $L_0=6$, $C_1=7$,  $C_2=8$,  $C_3=9$.** (Hay que tener en cuenta que las componentes de u y de p deben ser números de coma flotante, y no números enteros.)

> **Comprobación**
> Debería de obtenerse

          4-element Array{Float64,1}:
          3.0              
          4.0              
          48.91926581726429 
          -6.491838420715567

<a href="#top">Back to the top</a>

## 3 - Muelle menos rígido ($C_1 = 100$)

Supongamos que queremos estudiar la evolución de las variables de estado del péndulo con muelle para $C_1=100$, en el intervalo temporal $(0,40)$, y que no nos hace falta mucha precisión en los resultados. Para disminuir el tiempo de CPU, aplicaremos el método DP5 con tol=$10^{-4}$.

- **Para comenzar, antes de aplicar la función _solve_, hay que definir el problema.**


In [ ]:
u0= ?
p= ?
tspan = ?
prob = ODEProblem(fmuelle,u0,tspan,p);

 - **Aplicar DP5 con $tol=10^{-4}$, y representar gráficamente la evolución de las variables de estado con respecto del tiempo.**

- El algoritmo implementado en DP5() calcula las soluciones en una discretización del tiempo no uniforme. **Representar gráficamente la secuencia de longitudes de paso utilizadas a lo largo del intervalo temporal $[0,40]$.**  

In [ ]:
tt = ?
hh = ?
plot(tt[1:end-1],hh,title="Secuencia de longitudes de paso", xlabel="t", label="tol=1e-4", 
     seriestype=:scatter)

- Para comprobar la precisión de los resultados, **volver a aplicar DP5() al problema, pero esta vez con tol=$10^{-8}$.** Ahora, los resultados los calcula para otra discretización temporal no uniforme (más fina que la anterior, es decir, con más valores de los tiempos). **Para comprobarlo, comparar el número de instantes del tiempo considerados en los resultados obtenidos respectivamente con tol=$10^{-4}$ y con tol=$10^{-8}$.**

>**Comprobación**
>  
> En el caso tol=1e-4, resulta una discretización temporal con 252 valores del tiempo, 
> y en el caso tol=1e-8, en cambio, 1112. 

 - Consideraremos los resultados recién obtenidos como exactos (pues se supone que son mucho más precisos que los obtenidos con tol=$10^{-4}$). Hay que tener en cuenta que, aunque en cada uno de los dos casos (_sol_ para la solución numérica menos precisa, y _sol2_ para la solución "exacta") calcula los resultados para discretizaciones temporales distintas, se pueden calcular los valores "exactos" de las variables de estado en cualquier valor de $t$ dentro del intervalo temporal $[0,40]$. Para ello, basta con poner sol2(t) (el algoritmo aplica técnicas de interpolación para calcular una aproximación del vector de estados en t).
 **Como comprobación adicional, calcular los valores "exactos" de las variables de estado en $t=14.5$.**  

>**Comprobación**
>  
> Se deberían de obtener los siguientes valores de las variables de estado
>

           1.0030050443733272  
           1.4642087270483626  
          -0.016590545719411362
           0.4602876475898731                                 

 
 - **Definir una función errfun(t) que calcule la norma de los errores cometidos en el caso de tol=$10^{-4}$ en el tiempo t. Representar gráficamente la evolución de dicha norma de los errores en los tiempos de la discretización temporal de los resultados obtenidos para tol=$10^{-4}$.**

- **Utilizar @btime para medir y comparar los tiempos de ejecución para los resultados obtenidos con tol=$10^{-4}$ y con tol=$10^{-8}$ respectivamente.**

- **Comentar los resultados de tiempos de ejecución obtenidos.**
> 
> Escribir aquí los comentarios

<a href="#top">Back to the top</a>

## 4 - Muelle más rígido ($C_1 = 10000$)

### 4.1 - Método DP5

Ahora queremos volver a repetir los cálculos del punto anterior, pero para el caso en el que muelle sea más rígido, en concreto, con $C_1 = 10000$.


- **Para comenzar, antes de aplicar la función _solve_, hay que redefinir el problema.**


 - **Aplicar DP5 con $tol=10^{-4}$, y representar gráficamente la evolución de las variables de estado con respecto del tiempo, pero esta vez, cada variable de estado en una figura diferente** (debido a que tienen escalas de valores bastante diferentes).

- Para comprobar la precisión de los resultados, **volver a aplicar DP5() al problema, pero esta vez con tol=$10^{-8}$. Antes que nada, comparar el número de instantes del tiempo considerados en los resultados obtenidos respectivamente con tol=$10^{-4}$ y con tol=$10^{-8}$.**

- **Comentar el resultado**
>  
> Comentario

 
 - ** Representar gráficamente la evolución de la norma de los errores de los resultados obtenidos para tol=$10^{-4}$.**

- **Utilizar @btime para medir y comparar los tiempos de ejecución para los resultados obtenidos con tol=$10^{-4}$ y con tol=$10^{-8}$ respectivamente.**

- **Comentar los resultados de tiempos de ejecución obtenidos.**
> 
> Escribir aquí los comentarios

<a href="#top">Back to the top</a>

### 4.2 - Método RK implícito de Hairer de orden 4 (Hairer4)

- **Aplicar Hairer4() con $tol=10^{-4}$, y representar gráficamente la evolución de las variables de estado con respecto del tiempo,  cada variable de estado en una figura diferente**.

- Para comprobar la precisión de los resultados, **volver a aplicar Hairer4() al problema, pero esta vez con tol=$10^{-8}$. Antes que nada, comparar el número de instantes del tiempo considerados en los resultados obtenidos respectivamente con tol=$10^{-4}$ y con tol=$10^{-8}$.**

- **Comentar el resultado, comparandolo con lo ocurrido en el caso de DP5**
>  
> Comentario

 
 - ** Representar gráficamente la evolución de la norma de los errores de los resultados obtenidos para tol=$10^{-4}$.**

- **Utilizar @btime para medir y comparar los tiempos de ejecución para los resultados obtenidos con tol=$10^{-4}$ y con tol=$10^{-8}$ respectivamente.**

- **Comentar los resultados de tiempos de ejecución obtenidos, comparandolos con los obtenidos con DP5.**
> 
> Escribir aquí los comentarios

> Respuesta
>

**¿Que podemos decir en cuanto a la eficiencia de los dos métodos, en este ejemplo con $C_1=10000$?**

_Insertar aquí la respuesta._


<a href="#top">Back to the top</a>

### Valoración

_Incluir aquí los comentarios de valoración de la tarea a entregar (dificultad, interés, etc, incluidas, si se quiere, sugerencias de mejora del ejercicio), así como una estimación del tiempo dedicado al trabajo de la semana (desglosado en el tiempo de estudio de material teórico, tiempo de dedicación a la participación activa o pasiva en los foros, tiempo de implementación y experimentación con los problemas prácticos planteados, y tiempo de preparación del documento jupyter final)._